Hyper params

In [2]:
#!pip install git+https://github.com/huggingface/transformers

# The model that you want to train from the Hugging Face hub
model_name = "codellama/CodeLlama-7b-Instruct-hf"

# The instruction dataset to use
dataset_name = "pechaut/cairo-instruct"

#
# Fine-tuned model name
new_model = "llama-2-7b-cairo-test"

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

hub_name = "pechaut/codellama-cairo-instruct"

max_steps = 1000

Load Base Model and Tokenizer

In [3]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)


# load the quantized settings: 4 bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)

# don't use the cache
model.config.use_cache = False

# Load the tokenizer from the model (llama2)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

1

Import Dataset from Hugging Face

In [4]:
from datasets import load_dataset

# Load the dataset
dataset_train = load_dataset(dataset_name, split="train", download_mode='force_redownload',ignore_verifications=True)
dataset_test = load_dataset(dataset_name, split="eval", download_mode='force_redownload',ignore_verifications=True)


/home/pec/miniconda3/envs/cairo-code/lib/python3.9/site-packages/datasets/load.py:2097: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating eval split:   0%|          | 0/5 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/197 [00:00<?, ? examples/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating eval split:   0%|          | 0/5 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/197 [00:00<?, ? examples/s]

Check to see if everything is fine before launching training

In [5]:
df = dataset_train.to_pandas()
print(df["text"][18])

<s>[INST]In Cairo:  test if myArray is empty[/INST]
    if(myArry.is_empty) 'it is empty'.print()

 Explanations:

 - If you want to check if an array is empty or not, you can use the is_empty() method, which returns true if the array is empty and false otherwise.
 </s>


Load Model

In [6]:
from trl import SFTTrainer
import transformers
from peft import LoraConfig

qlora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

supervised_finetuning_trainer = SFTTrainer(
    model,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        max_steps=max_steps,
        logging_steps=10,
        output_dir="./SFTOpenLM-Dolly15k",
        optim="paged_adamw_8bit",
        fp16=True,
    ),
    tokenizer=tokenizer,
    peft_config=qlora_config,
    dataset_text_field="text",
    max_seq_length=512
)


/home/pec/miniconda3/envs/cairo-code/lib/python3.9/site-packages/peft/utils/other.py:133: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/197 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/home/pec/miniconda3/envs/cairo-code/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [7]:
supervised_finetuning_trainer.train()
supervised_finetuning_trainer.model.save_pretrained(new_model)
supervised_finetuning_trainer.tokenizer.save_pretrained(new_model)

You're using a CodeLlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,3.140900
20,2.067300
30,1.752600
40,1.602200
50,1.432800
60,1.379600
70,1.359900
80,1.098900
90,1.152700
100,1.110200


Load the old model, apply the new model and push to hub

In [29]:
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM
import torch
import os
model = AutoModelForCausalLM.from_pretrained(
            model_name, torch_dtype=torch.float16, load_in_8bit=False, device_map="auto", trust_remote_code=True)
     
peft_model = PeftModel.from_pretrained(model, new_model, from_transformers=True)
     

model = peft_model.merge_and_unload()

model.push_to_hub(hub_name)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(new_model)
tokenizer.push_to_hub(hub_name)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using pad_token, but it is not set yet.
Using pad_token, but it is not set yet.


CommitInfo(commit_url='https://huggingface.co/pechaut/codellama-cairo-instruct/commit/c7e337195258c59af4cdd70738639480b06e3a83', commit_message='Upload tokenizer', commit_description='', oid='c7e337195258c59af4cdd70738639480b06e3a83', pr_url=None, pr_revision=None, pr_num=None)

Define a pipe ready to use

In [31]:
from transformers import AutoTokenizer
from transformers import pipeline
import torch

tokenizer = AutoTokenizer.from_pretrained(hub_name)
pipe = pipeline(
    "text-generation",
    model=hub_name,
    torch_dtype=torch.float16,
    
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


RuntimeError: Error(s) in loading state_dict for LlamaForCausalLM:
	While copying the parameter named "model.layers.0.self_attn.q_proj.weight", whose dimensions in the model are torch.Size([4096, 4096]) and whose dimensions in the checkpoint are torch.Size([4096, 4096]), an exception occurred : ('Cannot copy out of meta tensor; no data!',).
	While copying the parameter named "model.layers.0.self_attn.k_proj.weight", whose dimensions in the model are torch.Size([4096, 4096]) and whose dimensions in the checkpoint are torch.Size([4096, 4096]), an exception occurred : ('Cannot copy out of meta tensor; no data!',).
	While copying the parameter named "model.layers.0.self_attn.v_proj.weight", whose dimensions in the model are torch.Size([4096, 4096]) and whose dimensions in the checkpoint are torch.Size([4096, 4096]), an exception occurred : ('Cannot copy out of meta tensor; no data!',).
	While copying the parameter named "model.layers.0.self_attn.o_proj.weight", whose dimensions in the mode

: 

In [13]:
prompt = "create a structure mailAccount, print it"



sequences = pipe(
   f"[INST]I'm working in Cairo. {prompt} [/INST]",
    do_sample=True,
    temperature=0.01,
    top_k=50,
    top_p=0.9,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: fn fibonacci(n: int) -> int:
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)


def fibonacci_memo(n: int) -> int:
    if n <= 1:
        return n
    else:
        return fibonacci_memo(n-1) + fibonacci_memo(n-2)


def fibonacci_memo_recursive(n: int, memo: dict = None) -> int:
    if memo is None:
        memo = {}
    if n <= 1:
        return n
    else:
        if n not in memo:
            memo[n] = fibonacci_memo_recursive(n-1, memo)


In [20]:
# Ignore warnings


#prompt = """Create an array array of some animal names and then print the size of it"""
#prompt="create an array containing some animal names, and print the length of the array, in cairo"
#prompt="in Cairo, define a u32"
#prompt="in cairo, write a 'hello world' program"
prompt = "create a structure mailAccount, print it"
#prompt = "create an array A containing 3  animal names. Create an array B, add all the elements of A to B, assert if the length of B is incorrect, create a span of B"
#prompt = "in cairo, create a program that computes the fibonacci sequence,give samples of the result "
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST]I'm working in Cairo. {prompt} [/INST]")
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST]I'm working in Cairo. create a structure mailAccount, print it [/INST]
    
struct MailAccount {
    name: u32,
    password: u32,
}

fn main() {
    let mut myAccount = MailAccount {
        name: 1,
        password: 2,
    };
    // do some stuff with myAccount
    std::mem::drop(myAccount);
    myAccount = MailAccount {
        name: 3,
        password: 4,
    };
    // do some stuff with myAccount again
}

Explanation:
- This program declares a struct MailAccount
- The first time myAccount is used, it is mut
- The second time myAccount is used, it is not mut

    
